In [1]:
%matplotlib inline

from __future__ import absolute_import

import numpy as np

import ghostpy.Invariants.LShell as ls
import ghostpy.algorithms.convert as algx
import ghostpy.algorithms.common as algc

import ghostpy.data.VtkData as vdt
import cProfile, pstats, StringIO
import ghostpy.data.GpData as gpd
import ghostpy.plotting.FieldLinePlot as flplt
import ghostpy.Invariants.FieldLine as fl
import random as rnd
import csv
from parse import *

import matplotlib.pylab as plt
import time
import glob

In [2]:
def pointTest(pos=None, data=None):
    timings = []
    lstars = []
    for locs in pos:
        loc = algc.sphere_to_cart(r=locs[0], lam=locs[1], phi=locs[2])
        start = time.time()
        lsl = ls.LShell(start_loc=loc, data=data, error_tol=2e-6)
        stop = time.time()
        dur = stop - start
        timings.append(dur)
        lstar = lsl.l_star(res=1000)
        lstars.append(lstar)
        print ("LStar: {} -- {}".format(lstar, dur))
        
    return np.array(timings), np.array(lstars)

In [3]:
def convergeTest(pos=None, data=None):
    for loc in pos:
        lsl = ls.LShell(start_loc=loc, data=data, save_lines=True, error_tol=2e-6)
        lstar = lsl.l_star(res=10000)
        print ("LStar: {}".format(lstar))
        lsl.converge_lstar(tol=1e-3)
        lstar = lsl.l_star(res=1000)
        print ("LStar(C): {}".format(lstar))

In [4]:
def convergeTest2(pos=None, data=None):
    for loc in pos:
        lsl = ls.LShell(start_loc=loc, data=data, save_lines=True, error_tol=2e-6)
        lstar = lsl.l_star(res=10000)
        print ("LStar: {}".format(lstar))
        lsl.converge_p2(depth=3)
        lstar = lsl.l_star(res=10000)
        print("LStar(C): {}".format(lstar))

In [5]:
def process_stats(prof_lines=None, timeLine=None, keyLine=None, dataStartLine=None):
    assert prof_lines is not None
    assert timeLine is not None
    assert keyLine is not None
    assert dataStartLine is not None
    count = 0
    profile = {}
    keys = []
    pstring = "{file}:{line}({function})"
    for line in prof_lines:
        # print ("Count: {}".format(count))
        ls_prof = {}
        if count == timeLine:
            # print ("line: {}".format(count))
            # print ("{}".format(line))
            stats = line.split()
            time = stats[-2]
            profile['time'] = time
        elif count == keyLine:
            # print ("line: {}".format(count))
            # print ("{}".format(line))
            keys = line.split()
            # print ("Keys: {}".format(keys))
        elif count >= dataStartLine:
            # print ("line: {}".format(count))
            # print ("{}".format(line))
            stats = line.split()
            # print (stats)
            # print (len(stats))
            if len(stats) == 6:
                ls_prof[keys[0]] = stats[0]
                ls_prof[keys[1]]=stats[1]
                ls_prof[keys[2]]=stats[2]
                ls_prof[keys[3]]=stats[3]
                ls_prof[keys[4]]=stats[4]
                filestats = parse(pstring, stats[5])
                
                try:
                    profile[filestats['file']][filestats['function']] = ls_prof
                except:
                    try:
                        profile[filestats['file']] = {}
                        profile[filestats['file']][filestats['function']] = ls_prof
                    except:
                        pass
        count += 1
    
    return profile

In [6]:
outpath = "/Volumes/8TB Seagate/PhD Data/profiles/"
data = vdt.VtkData(filename="../unit_tests/test_data/WHIQuad.vts", vector='B')

In [ ]:
r = np.linspace(start=2.4, stop=8.5, num=5000)
lam = np.linspace(start=-np.pi/4, stop=np.pi/4, num=5000)
phi = np.linspace(start=0, stop=np.pi*2, num=5000, endpoint=False)

cnt = 50

rs = np.random.choice(r, cnt)
lams = np.random.choice(lam, cnt)
phis = np.random.choice(phi, cnt)

ps = np.vstack([rs,lams,phis]).T

In [ ]:
profile = cProfile.Profile()
profile.enable()
timings, lstars = pointTest(pos=ps, data=data)
profile.disable()

# ps1.print_stats()




VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point (-2.2329414817162885, -1.5798407917364792, -0.47970008152713062)
K-list: [  9.98843136  10.01015967  10.39513869]
B_mirror = 1467.32282932
K = 10.3951386917
Start Phi: 3.75734481369
Dipole L: 2.86328907969


/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/Users/jjm390/src/ghostpy/ghostpy/ghostpy/Invariants/FieldLine.py:689: RuntimeWarning: All-NaN slice encountered
  n_k = np.nanmax([kf,kb], axis=0)


Inner Gap: 10.52646734
Outer Gap: -3.80052481879
Iterations to converge: 27
...
Inner Gap: 1.93125420918
Outer Gap: -0.276252615484
Iterations to converge: 27
...
Inner Gap: 3.44462367655
Outer Gap: -1.0295829155
Iterations to converge: 27
...
Inner Gap: 2.72327193942
Outer Gap: -19.6760970223
Iterations to converge: 27
...
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 2.83132746038 -- 5.71156191826



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point (-4.1572925494080586, -3.1583023693624881, 2.990646666410298)
K-list: [ 93.41004007  99.75878955  99.76435528]
B_mirror = 200.901498945
K = 99.7643552779
Start Phi: 3.79127401435
Dipole L: 8.24623436873
Inner Gap: 0.148974410781
Outer Gap: -0.0241747120934
Iterations to converge: 29
...
Inner Gap: 1.59288928345
Outer Gap: -0.0615213911662
Iterations to converge: 29
...
Inner Gap: 0.978725769974
Outer Gap: -0.369977929539
Iterations to converge: 29
...
Inner Gap: 1.14657904476
Outer Gap: -2

LStar: 7.78882047413 -- 31.2775859833



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point (2.7483102561175539, -1.1770189324545559, -0.34964520670236571)
K-list: [ 5.10341186  5.16441924  6.97854479]
B_mirror = 1153.54302446
K = 6.97854478794
Start Phi: 5.8785481734
Dipole L: 3.06060717809
Inner Gap: 0.679812812849
Outer Gap: -0.268435095306
Iterations to converge: 27
...
Inner Gap: 0.470234915138
Outer Gap: -2.19445281013
Iterations to converge: 27
...
Inner Gap: 4.89957689927
Outer Gap: -0.322546222576
Iterations to converge: 27
...
Inner Gap: 4.05819392603
Outer Gap: -10.4331761657
Iterations to converge: 27
...
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 3.04502947326 -- 6.35479283333



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point (1.2160676331228897, -3.0379253698462256, 0.31197742657480604)
K-list: [ 1.20108873  2.87340735  3.03232393]
B_mirror = 870.29252554
K = 3.03232392816
St

Inner Gap: 0.0717826926584
Outer Gap: -0.137540422736
Iterations to converge: 28
...
Inner Gap: 0.143625292047
Outer Gap: -0.159650685787
Iterations to converge: 28
...
Inner Gap: 0.00442647445826
Outer Gap: -0.181797572603
Iterations to converge: 28
...
Inner Gap: 0.16105910683
Outer Gap: -0.0107285009392
Iterations to converge: 28
...
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 6.98275497281 -- 26.6955139637



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point (-0.93365887903899458, -5.8473236499694874, 0.5795667246228251)
K-list: [ 1.61941581  3.48549655  3.63274635]
B_mirror = 153.786544456
K = 3.63274635155
Start Phi: 4.55405271064
Dipole L: 6.11569704343
Inner Gap: 0.0283197379098
Outer Gap: -0.0127135476884
Iterations to converge: 28
...
Inner Gap: 0.0165434284438
Outer Gap: -0.0318570365126
Iterations to converge: 28
...
Inner Gap: 0.282670684743
Outer Gap: -0.156379428537
Iterations to converge: 28
...
Inner Gap: 0.0154325517

In [ ]:
s1 = StringIO.StringIO()
ps1 = pstats.Stats(profile, stream=s1)
ps1 = ps1.strip_dirs()
ps1.print_stats()
prof = s1.getvalue()
prof_lines1 = prof.split('\n')

profileDict = process_stats(prof_lines1, timeLine=0, keyLine=4, dataStartLine=6)

In [ ]:
fig = plt.figure(figsize=(10, 8), facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
lprofile = profileDict['LShell.py']
y = []
y2 = []

xlab = []

n = float(lprofile['__init__']['ncalls'])
t = float(lprofile['__init__']['cumtime'])/n
for key in lprofile:
    if key == 'add_trace' or key == '__init__' or key == '__trace_from_location__' or key == 'l_star':
        xlab.append(key)
        y.append((float(lprofile[key]['cumtime'])/n)/t*100)

for key in lprofile:
    if key == 'add_trace' or key == '__init__' or key == '__trace_from_location__' or key == 'l_star':
        y2.append((float(lprofile[key]['cumtime'])/float(lprofile[key]['ncalls']))/t*100)

idx = np.arange(len(xlab))
idx_name = xlab

ax.bar(idx, y, align='center')
plt.xticks(idx, idx_name, rotation='vertical')

ax.set_ylabel("Time (%)")
ax.set_yscale('log')
ax.set_title("Average Execution Time per Method Call for a 4-point Converged $L^*$ Calculation\n For a Single Point")
ax.set_xlabel("Function Call")
s2 = StringIO.StringIO()
ps2 = pstats.Stats(profile, stream=s2)
ps2.print_stats('FieldLine.py:')
plt.grid(b=True, which='major', color='grey', linewidth=0.25, linestyle=':')

rects = ax.patches

# Now make some labels
labels = ["{:.3f} %".format(i) for i in y]
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height/1.3, label, ha='center', va='bottom')

ax.bar(idx, y2, align='center', color='orange', width=rect.get_width()/1.5)

labels = ["{:.3f} %".format(i) for i in y2]
rects = ax.patches
for rect, label in zip(rects[4:], labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height/1.3, label, ha='center', va='bottom')
    if label == labels[3]:
        ax.text(rect.get_x() + rect.get_width()/2, height/1.6, "{:.2f} s".format(t), ha='center', va='bottom')


prof2 = s2.getvalue()
prof_lines = prof.split('\n')
fig.tight_layout()
fig.savefig("/Volumes/8TB Seagate/PhD Data/profiles/GHOSTpyProfile1pt.pdf")
# fig.savefig("/home/jmurphy/profiles/GHOSTpyProfile.pdf")

In [ ]:
fig = plt.figure(figsize=(10, 8), facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
lprofile = profileDict['FieldLine.py']
y = []
y2 = []

xlab = []

lprofile = profileDict['LShell.py']

searchCount = float(lprofile['__search_B_adaptive__']['ncalls'])
t = float(lprofile['__search_B_adaptive__']['cumtime'])/searchCount

for key in lprofile:
    if key == 'get_raw_path' or key == '__search_B_adaptive__':
        xlab.append(key)
        y.append((float(lprofile[key]['cumtime'])/float(searchCount))/t*100)
        
lprofile = profileDict['FieldLine.py']
for key in lprofile:
    if key == '__trace_line__' or key == '__get_min_b_gap__':
        xlab.append(key)
        y.append((float(lprofile[key]['cumtime'])/float(searchCount))/t*100)
        
lprofile = profileDict['LShell.py']

for key in lprofile:
    if key == 'get_raw_path' or key == '__search_B_adaptive__':
        y2.append((float(lprofile[key]['cumtime'])/float(lprofile[key]['ncalls']))/t*100)
        
lprofile = profileDict['FieldLine.py']
for key in lprofile:
    if key == '__trace_line__' or key == '__get_min_b_gap__':
        y2.append((float(lprofile[key]['cumtime'])/float(lprofile[key]['ncalls']))/t*100)


idx = np.arange(len(xlab))
idx_name = xlab

ax.bar(idx, y, align='center')
plt.xticks(idx, idx_name, rotation='vertical')

ax.set_ylabel("Time (%)")
ax.set_yscale('log')
ax.set_title("Average Execution Time per Method Call for KB Line Search")
ax.set_xlabel("Function Call")
s2 = StringIO.StringIO()
ps2 = pstats.Stats(profile, stream=s2)
ps2.print_stats('FieldLine.py:')
plt.grid(b=True, which='major', color='grey', linewidth=0.25, linestyle=':')

rects = ax.patches

# Now make some labels
labels = ["{:.4f} %".format(i) for i in y]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height/1.6, label, ha='center', va='bottom')

ax.bar(idx, y2, align='center', color='orange', width=rect.get_width()/1.5)

labels = ["{:.4f} %".format(i) for i in y2]
rects = ax.patches
for rect, label in zip(rects[4:], labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height/1.6, label, ha='center', va='bottom')
    if label == labels[1]:
        ax.text(rect.get_x() + rect.get_width()/2, height/2.3, "{:.4f} s".format(t), ha='center', va='bottom')




prof2 = s2.getvalue()
prof_lines = prof.split('\n')
fig.tight_layout()
fig.savefig("/Volumes/8TB Seagate/PhD Data/profiles/GHOSTpyProfileKBSearch.pdf")
# fig.savefig("/home/jmurphy/profiles/GHOSTpyProfile.pdf")

In [ ]:
fig = plt.figure(figsize=(10, 8), facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
y = []
y2 = []

xlab = []

lprofile = profileDict['FieldLine.py']
searchCount = float(lprofile['__init__']['ncalls'])
t = float(lprofile['__init__']['cumtime'])/searchCount


keyList = ['__init__',  '__k_mod__', '__trace_line__']

for key in lprofile:
    if key in keyList:
        xlab.append(key)
        y.append((float(lprofile[key]['cumtime'])/float(searchCount))/t*100)
                
for key in lprofile:
    if key in keyList:
        y2.append((float(lprofile[key]['cumtime'])/float(lprofile[key]['ncalls']))/t*100)


idx = np.arange(len(xlab))
idx_name = xlab

ax.bar(idx, y, align='center')
plt.xticks(idx, idx_name, rotation='vertical')

ax.set_ylabel("Time (%)")
ax.set_yscale('log')
ax.set_title("Average Execution Time per Method Call for Field Line Trace and $K$ Model")
ax.set_xlabel("Function Call")
s2 = StringIO.StringIO()
ps2 = pstats.Stats(profile, stream=s2)
ps2.print_stats('FieldLine.py:')
plt.grid(b=True, which='major', color='grey', linewidth=0.25, linestyle=':')

rects = ax.patches

# Now make some labels
labels = ["{:.3f} %".format(i) for i in y]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height/1.10, label, ha='center', va='bottom')

ax.bar(idx, y2, align='center', color='orange', width=rect.get_width()/1.5)

labels = ["{:.3f} %".format(i) for i in y2]
rects = ax.patches
for rect, label in zip(rects[3:], labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height/1.10, label, ha='center', va='bottom')
    if label == labels[2]:
        ax.text(rect.get_x() + rect.get_width()/2, height/1.17, "{:.2f} s".format(t), ha='center', va='bottom')


prof2 = s2.getvalue()
prof_lines = prof.split('\n')
fig.tight_layout()
fig.savefig("/Volumes/8TB Seagate/PhD Data/profiles/GHOSTpyProfileFLine.pdf")
# fig.savefig("/home/jmurphy/profiles/GHOSTpyProfile.pdf")

In [ ]:
wnan = ~np.isnan(lstars)
lstars2 = lstars[np.where(wnan)]
timings2 = timings[np.where(wnan)]

In [ ]:
fig = plt.figure(figsize=(10, 5), facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax.set_title("Histogram of Calculated $L^*$ Values")
ax.set_xlabel("$L^*$")

ax.hist(lstars2, bins='auto')
plt.grid(b=True, which='major', color='grey', linewidth=0.25, linestyle=':')
plt.tight_layout()

fig.savefig("/Volumes/8TB Seagate/PhD Data/profiles/histL.pdf")

In [ ]:
fig = plt.figure(figsize=(10, 5), facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax.set_title("Histogram of Execution Time for Calculated $L^*$ Values")
ax.set_xlabel("Execution Time (s)")

ax.hist(timings2, bins='auto')
plt.grid(b=True, which='major', color='grey', linewidth=0.25, linestyle=':')
plt.tight_layout()
fig.savefig("/Volumes/8TB Seagate/PhD Data/profiles/histT.pdf")

In [ ]:
np.nanmedian(timings2)

In [ ]:
np.nanmean(timings2)

In [ ]:
import pickle

In [ ]:
pickle.dump(lstars2, open("/Volumes/8TB Seagate/PhD Data/profiles/lstarsProfile.pickle", "wb"))
pickle.dump(timings2, open("/Volumes/8TB Seagate/PhD Data/profiles/timingsProfile.pickle", "wb"))
pickle.dump(profileDict, open("/Volumes/8TB Seagate/PhD Data/profiles/profileDictionary.pickle", "wb"))

In [ ]:
fig = plt.figure(figsize=(10, 5), facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax.scatter(lstars2, timings2, marker="+", color='k')
ax.set_xlabel("$L^*$")
ax.set_ylabel("Execution Time (s)")
ax.set_title("Particle $L^*$ vs. Execution Time (4-Line Convergence)\nQuad Res LFM Grid")
plt.grid(b=True, which='major', color='grey', linewidth=0.25, linestyle=':')
plt.tight_layout()
fig.savefig("/Volumes/8TB Seagate/PhD Data/profiles/LvTime.pdf")

In [ ]:
import pickle
